# Carrier Phase Offset
Carrier phase offset is a randomized phase between 0 and $2\pi$ that is applied to a signal. The transform is usually employed to simulate a fixed carrier phase offset corresponding to a fixed propagation time delay for the signal that may include transmitter hardware, the channel, and the receiver hardware. The effect is applied by multiplying the signal by complex exponential $e^{j\phi}$ where $\phi \sim U(0,2\pi)$.

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
import torchsig.transforms.functional as F

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Function for generating the test QPSK modulated signal.

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

Generate modulated input data and apply the transform with a fixed pi/8 phase offset.

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data

# apply transform
impaired_qpsk_data = F.phase_offset(
    data = qpsk_data,
    phase = np.pi/8
)

Plot the I/Q constellation result - note the fixed pi/8 rotation for all samples.

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
ax.set_box_aspect(1)
ax.plot(np.real(qpsk_data),np.imag(qpsk_data),label='Input')
ax.plot(np.real(impaired_qpsk_data),np.imag(impaired_qpsk_data),alpha=0.5,label='Phase Offset')
ax.grid()
ax.set_title('Constellation Plot')
ax.set_xlabel('Real')
ax.set_ylabel('Imag')
ax.legend(fontsize='large', loc='upper left');